In [1]:
import requests
import pandas as pd
import sys, os
import ta
import time
import hmac, hashlib
from ta.utils import dropna


import okx.Trade as Trade
import okx.PublicData as PublicData
import okx.Account as Account

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
def get_okx_kline_data(symbol, interval, limit=100):
    base_url="https://www.okx.com/api/v5/market/history-candles"
    params={
        "instId":symbol,
        "bar":interval,
        "limit":limit
    }
    try:
        response = requests.get(base_url,params=params)
        response.raise_for_status()
        return response
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"Failed to fetch data for {symbol}: {e}")
        return None

In [6]:
def generate_signals(df):
    df=df.reset_index(drop=True)

    #Initialize the variables
    df['buy_signal']=False
    df['sell_signal']=False

    #Buy when MACD crosses above the MACD signal and RSI is below 30 (oversold), and price touches lower Bollinger Band
    #Sell when MACD crosses below the MACD signal and RSI is above 70 (overbought), and price touches upper Bollinger Band
    for i in range(1, len(df)):
        if(df['macd'][i]> df['macd_signal'][i]) and (df['macd'][i-1] <= df['macd_signal'][i-1]) and (df['rsi'][i]<30) and (df['close'][i]<=df['bb_low'][i]):
            df.at[i, 'buy_signal'] = True
        elif (df['macd'][i] < df['macd_signal'][i]) and (df['macd'][i-1] >= df['macd_signal'][i-1]) and (df['rsi'][i]>70) and (df['close'][i] >= df['bb_high'][i]):
            df.at[i, 'sell_signal'] = True
    return df

def get_signals_for_pair(pair, interval="1m", limit=100):
    response = get_okx_kline_data(pair, interval, limit)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Failed to fetch data for {pair}")
        return None
    raw_data = data["data"]
    return raw_data
    df = process_data(raw_data)
    df_with_indicators = add_indicators(df)
    df_with_signals = generate_signals(df_with_indicators)    
    return df_with_signals[["timestamp","close","buy_signal","sell_signal"]]

def process_data(data):
    df =  pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume", "u1", "u2", "u3"])
    df["timestamp"]= pd.to_datetime(df["timestamp"], unit="ms")

    #Convert columns to numeric data types
    for col in ["open", "high", "low", "close", "volume"]:
        df[col]=pd.to_numeric(df[col], errors='coerce')

    #Clean NaN values
    df = dropna(df)

    #Add all ta features
    #df = add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume")
    df = add_indicators(df)

    #Keep only the columns we're interested in
    # old version df = df[["timestamp", "open", "high", "low", "close", "volume", "momentum_rsi", "volatility_bbh", "volatility_bbl", "trend_macd", "trend_macd_signal", "trend_macd_diff"]]
    df = df[["timestamp", "open", "high", "low", "close", "volume", "rsi", "bb_high", "bb_low", "macd", "macd_signal", "macd_diff"]]      
    return df

def add_indicators(df):
    #RSI
    df["rsi"] = ta.momentum.RSIIndicator(close=df["close"], window=14, fillna=False).rsi()

    #Bollinger Bands
    bb = ta.volatility.BollingerBands(close=df["close"], window=20)
    df["bb_high"] = bb.bollinger_hband()
    df["bb_low"] = bb.bollinger_lband()

    #MACD
    macd=ta.trend.MACD(close=df["close"], window_slow=26, window_fast=12, window_sign=9)
    df["macd"]=macd.macd()
    df["macd_signal"]=macd.macd_signal()
    df["macd_diff"]=macd.macd_diff()

    #print(df)
    return df

In [7]:
def get_available_trading_pairs():
    PublicDataAPI = PublicData.PublicAPI(flag="1")
    resultPublicData = PublicDataAPI.get_instruments(instType="SPOT")
    if 'data' in resultPublicData:
        live_pairs = [pair for pair in resultPublicData['data'] if pair.get('state') == 'live']
        return live_pairs
    else:
        return []
    
import okx.MarketData as MarketData
def get_available_market_data():
    flag = "1"  # Production trading:0 , demo trading:1
    marketDataAPI =  MarketData.MarketAPI(flag=flag)
    # Retrieve the latest price snapshot, best bid/ask price, and trading volume in the last 24 hours
    result = marketDataAPI.get_tickers(
        instType="SPOT"
    )
    return result



In [12]:
crypto_pairs_full = get_available_market_data()
if 'data' in crypto_pairs_full:
        crypto_pairs = [pair for pair in crypto_pairs_full['data']]
"""print(crypto_pairs)
for pair_info in crypto_pairs:
    print(pair_info['instId'])"""

#for pair_info in crypto_pairs:
#        pair = pair_info['instId']
#        print(pair)
get_signals_for_pair("BTC-USDT", "1m", 100)

header:  {'Content-Type': 'application/json', 'x-simulated-trading': '1'}
domain: https://www.okx.com
url: /api/v5/market/tickers?instType=SPOT


[['1715459040000',
  '61080.4',
  '61080.4',
  '61077.9',
  '61077.9',
  '0.02258599',
  '1379.551829571',
  '1379.551829571',
  '1'],
 ['1715458980000',
  '61090.1',
  '61090.1',
  '61080.4',
  '61080.5',
  '1.26439845',
  '77236.785535779',
  '77236.785535779',
  '1'],
 ['1715458920000',
  '61100',
  '61100',
  '61090',
  '61090',
  '0.41014347',
  '25058.356952822',
  '25058.356952822',
  '1'],
 ['1715458860000',
  '61114.5',
  '61114.5',
  '61100',
  '61100',
  '0.10073416',
  '6155.198011438',
  '6155.198011438',
  '1'],
 ['1715458800000',
  '61139.9',
  '61139.9',
  '61114.6',
  '61114.6',
  '0.17612509',
  '10764.681715932',
  '10764.681715932',
  '1'],
 ['1715458740000',
  '61122.7',
  '61145.2',
  '61122.7',
  '61142.8',
  '0.3282699',
  '20067.641240014',
  '20067.641240014',
  '1'],
 ['1715458680000',
  '61109.9',
  '61118.2',
  '61109.9',
  '61118.2',
  '0.17939199',
  '10963.230384629',
  '10963.230384629',
  '1'],
 ['1715458620000',
  '61089.6',
  '61110',
  '61089.6',
  